# Comparison between patients with short and long disease

In this notebook we give summary statistics for the whole cohort of PD patients stratified by the length of the disease. The first group are those who have been diagnosed less than 5 years before the assessment and the second group are those who have been diagnosed more than 5 years before the assessment.

In [1]:
def get_library_path()->str:

    cwd = os.getcwd()
    lst = cwd.split(os.sep)[:-1]

    path_lib = f'{os.sep}'

    for elem in lst:
        path_lib = os.path.join(path_lib, elem)

    return path_lib

# imports

import os
import sys

path_to_lib = get_library_path()

sys.path.append(path_to_lib)
import luxgiant_clinical.TwoCatAnalysis as two
from luxgiant_clinical.Helpers import categories_recoder

import pandas as pd
import numpy as np

In [2]:
# load data

folder_path = os.path.join(path_to_lib, 'data/source')

df = pd.read_csv(
    os.path.join(folder_path, 'cleaned_file.csv'), 
    low_memory=False
)

In [3]:
# filter data to keep only patients with duration of PD information

mask_patients= (df['Status']=='Patient')
mask_duration= (~df['PD_duration'].isnull()) 

df_cases = df[mask_patients & mask_duration].reset_index(drop=True)
del df

df_cases.shape

(7473, 716)

## Summary statistics for demography and lifestyle

In [4]:
# rename and select first group of variables

variables_dict_1 = {
    "age"                       : "Age at Recruitment (years)",
    "age_at_onset"              : "Age at Onset (years)" ,
    "sex"                       : "Male" ,
    "agecat_1"                  : "Onset <50 years",
    "agecat_2"                  : "Onset <40 years",
    "PD_duration"               : "Duration of Disease (years)",
    "years_of_education"        : "Years of Education",
    "bmi_comp"                  : "Body Mass Index",
    "reason"                    : "PD-related Reason for Stop Working",
    "work"                      : "Currently Employed",
    "nature_of_work___1"        : "Agriculture as a job",
    "over_your_lifetime_have_yo": "Life time direct exposure to pesticide/insecticide/fungicide",
    "during_your_lifetime_did_y": "Exposure to chemicals at home",
    "in_your_lifetime_have_you" : "Smoked 100 or more cigarettes in lifetime",
    "in_your_lifetime_have"     : "Regular consumption of caffeinated drinks for >6months",
    "have_you_ever_had_a_head_i": "Head injury or concussion",
}
variables_1 = ['participant_id', 'pdsl'] + list(variables_dict_1.keys())

# statistical measures
stats_meas = {
    'mean'  : ["Age at Recruitment (years)", "Age at Onset (years)", "Body Mass Index"],
    'n'     : ["Male", "Onset <50 years", "Onset <40 years", "PD-related Reason for Stop Working", "Currently Employed", 
               "Agriculture as a job", "Life time direct exposure to pesticide/insecticide/fungicide", 
               "Exposure to chemicals at home",
               "Smoked 100 or more cigarettes in lifetime", "Regular consumption of caffeinated drinks for >6months",
               "Head injury or concussion"],
    'median': ["Duration of Disease (years)", "Years of Education"]
}

groups = ['PD duration <=5 yrs', 'PD duration >5 yrs']

In [5]:
# keep only variables for analysis

df_cases_1 = df_cases[variables_1].copy()
df_cases_1 = df_cases_1.rename(columns=variables_dict_1)

In [6]:
# recode variable values

recode_dict = {
    "Currently Employed"                :{'Yes':1, 'No':0},
    "Agriculture as a job"              :{'Checked':1, 'Unchecked':0},
    "Onset <50 years"                   :{'<50':1, '>=50':0},
    "Onset <40 years"                   :{'<40':1, '>=40':0},
    "PD-related Reason for Stop Working":{'PD related':1, 'not PD related':0},
    "Male"                              :{'Male':1, 'Female':0},
    "pdsl"                              :{"<=5":'PD duration <=5 yrs', ">5":'PD duration >5 yrs'}
}
for key in recode_dict.keys():
    df_cases_1 = categories_recoder(df_cases_1, [key], recode_dict[key])

recode = ["Life time direct exposure to pesticide/insecticide/fungicide", 
          "Exposure to chemicals at home", "Smoked 100 or more cigarettes in lifetime",
          "Regular consumption of caffeinated drinks for >6months", "Head injury or concussion"]
mapping = {'Yes':1, 'No':0, 'Dont Know':np.nan, 'Refused':np.nan, 'Possibly':np.nan}

df_cases_1 = categories_recoder(df_cases_1, recode, mapping)

In [7]:
summary = [
    two.report_mean_std(
        data_df    =df_cases_1,
        variables  =stats_meas['mean'],
        groups     =groups,
        grouping_by='pdsl'
    ),
    two.report_median_iqr(
        data_df    =df_cases_1,
        variables  =stats_meas['median'],
        groups     =groups,
        grouping_by='pdsl'
    ),
    two.report_proportion(
        data_df    =df_cases_1,
        variables  =stats_meas['n'],
        groups     =groups,
        grouping_by='pdsl'
    )
]

# Analysis of patients medical history

In [8]:
# rename and select second group of variables

variables_dict_2 = {
    "medical_history_neurologic___7": "Stroke",
    "medical_history_cancer___1"    : "Melanoma" ,
    "medical_history_cancer___2"    : "Prostate Cancer" ,
    "medical_history_cancer___3"    : "Other Cancers",
    "medical_history_metabolic___1" : "Diabetes",
    "medical_history_metabolic___3" : "High Cholesterol",
    "medical_history_cardiovasc___2": "Hypertension",
    "medical_history_cardiovasc___3": "History of CAD",
    "medical_history_psychologi___1": "History of Anxiety",
    "medical_history_psychologi___3": "History of Depression",
    "family_member_diagnosed_wi"    : "Family History of PD",
    "family_member_diagnosed_wi_2"  : "Family History of Tremor",
    "family_member_diagnosed"       : "Family History of Dementia"
}
variables_2 = ['participant_id', 'pdsl'] + list(variables_dict_2.keys())

# statistical measures
stats_meas2 = {
    'n'     : ["Stroke", "Melanoma", "Prostate Cancer", "Other Cancers", "Diabetes", "High Cholesterol", "Hypertension",
               "History of CAD", "History of Anxiety", "History of Depression", "Family History of PD", "Family History of Tremor", "Family History of Dementia"]
}

In [9]:
# keep only variables for analysis

df_cases_2 = df_cases[variables_2].copy()
df_cases_2 = df_cases_2.rename(columns=variables_dict_2)

In [10]:
# recode variable values

df_cases_2 = categories_recoder(df_cases_2, ["pdsl"], {"<=5":'PD duration <=5 yrs', ">5":'PD duration >5 yrs'})

fam_hist = ["Family History of PD", "Family History of Tremor", "Family History of Dementia"]
remaining= [feat for feat in stats_meas2['n'] if feat not in fam_hist]

df_cases_2 = categories_recoder(df_cases_2, fam_hist, {"Yes":1, "No":0})
df_cases_2 = categories_recoder(df_cases_2, remaining, {"Checked":1, "Unchecked":0})

In [11]:
# count and proportion of categorical variables

df_sum2 = two.report_proportion(
    data_df    =df_cases_2,
    variables  =stats_meas2['n'],
    groups     =groups,
    grouping_by='pdsl',
    subheader  ='Medical History'
)

In [12]:
# save result in csv file

df_7 = pd.concat( summary+ [df_sum2], axis=0, ignore_index=True)

df_7 = two.final_formatter(df_7, groups)
df_7.to_csv(os.path.join(path_to_lib, 'data/auxiliar/aux_table_7.csv'), index=False)
df_7

,Variable,Statistical Measure,PD duration <=5 yrs,PD duration >5 yrs,p-value,Total,Available Samples for Analysis
0,Age at Recruitment (years),mean (SD),59.5 (11.7),61.4 (10.6),p<0.001,60.4 (11.3),7468
1,Age at Onset (years),mean (SD),56.8 (11.8),51.0 (11.0),p<0.001,54.2 (11.8),7468
2,Body Mass Index,mean (SD),24.3 (3.6),24.2 (3.7),0.1301,24.3 (3.6),6670
3,Duration of Disease (years),median (IQR),3.0 (1.0 - 4.0),10.0 (7.0 - 12.0),p<0.001,5.0 (2.0 - 9.0),7473
4,Years of Education,median (IQR),12.0 (8.0 - 15.0),12.0 (10.0 - 15.0),p<0.001,12.0 (9.0 - 15.0),7095
5,Male,n (%),2807 (67.7),2259 (67.9),0.8165,5066 (67.8),7472
6,Onset <50 years,n (%),1052 (25.4),1398 (42.0),p<0.001,2450 (32.8),7468
7,Onset <40 years,n (%),351 (8.5),482 (14.5),p<0.001,833 (11.2),7468
8,PD-related Reason for Stop Working,n (%),640 (28.9),769 (38.7),p<0.001,1409 (33.6),4197
9,Currently Employed,n (%),2122 (52.9),1515 (47.1),p<0.001,3637 (50.3),7229
